In [29]:
import tkinter as tk
from tkinter import messagebox

# ==========================================
# 1. KLASSEN UND HILFSFUNKTIONEN
# ==========================================

class Spielstein:
    """Repräsentiert ein Haus, Kirchenschiff oder Kirchturm."""
    def __init__(self, spieler_id, typ, ausrichtung='N'):
        self.spieler = spieler_id
        self.typ = typ             # 'Haus', 'Schiff', 'Turm'
        self.ausrichtung = ausrichtung

    def drehen(self):
        """Dreht den Stein nach einem Zug um 90 Grad im Uhrzeigersinn."""
        # Zyklus: N -> O -> S -> W -> N
        richtungen = ['N', 'O', 'S', 'W']
        if self.ausrichtung in richtungen:
            idx = richtungen.index(self.ausrichtung)
            self.ausrichtung = richtungen[(idx + 1) % 4]

    def get_farbe(self):
        # Helle Farben für besseren Kontrast zu den schwarzen Pfeilen
        return '#FF9999' if self.spieler == 1 else '#9999FF'

def get_bewegungs_vektor(ausrichtung):
    """Wandelt Ausrichtung in Delta-Koordinaten um."""
    if ausrichtung == 'N': return (0, -1)
    if ausrichtung == 'O': return (1, 0)
    if ausrichtung == 'S': return (0, 1)
    if ausrichtung == 'W': return (-1, 0)
    return (0, 0)

# --- KONSTANTEN ---
ROWS, COLS = 7, 7
CELL_SIZE = 90
WINDOW_SIZE = ROWS * CELL_SIZE
LINE_WIDTH = 3
OFFSET = 30

# ==========================================
# 2. POPUP FÜR RICHTUNGSWAHL
# ==========================================
class RichtungsWaehler(tk.Toplevel):
    """Fenster, das den Spieler fragt, wie der Stein ausgerichtet werden soll."""
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Ausrichtung wählen")
        self.geometry("250x200")
        self.ergebnis = None

        # Zentrieren über dem Hauptfenster
        x = parent.winfo_x() + 50
        y = parent.winfo_y() + 50
        self.geometry(f"+{x}+{y}")

        tk.Label(self, text="Wie soll der Stein stehen?", font=('Arial', 12)).pack(pady=20)

        btn_font = ('Arial', 10, 'bold')
        # Wir setzen intern N für Vertikal und O für Horizontal als Startwert
        tk.Button(self, text="↕ Vertikal (Nord/Süd)",
                  command=lambda: self.setze('N'), width=20, font=btn_font).pack(pady=5)

        tk.Button(self, text="↔ Horizontal (Ost/West)",
                  command=lambda: self.setze('O'), width=20, font=btn_font).pack(pady=5)

        # Fenster modal machen (blockiert das Hauptspiel bis zur Wahl)
        self.transient(parent)
        self.grab_set()
        self.wait_window(self)

    def setze(self, richtung):
        self.ergebnis = richtung
        self.destroy()

# ==========================================
# 3. HAUPTKLASSE SPIELBRETT
# ==========================================

class Spielbrett(tk.Canvas):

    def __init__(self, master):
        super().__init__(master, width=WINDOW_SIZE + 2 * OFFSET, height=WINDOW_SIZE + 2 * OFFSET, bg='#F0F0F0')
        self.pack()

        # Datenstrukturen
        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]
        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1),
            2: self._initialisiere_steine(2)
        }
        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spieler_farben = {1: 'Rot', 2: 'Blau'}

        # Start
        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld() # Setzt nur den Pfarrer

        self.bind("<Button-1>", self.handle_click)
        self.update_title()

    def _initialisiere_steine(self, spieler_id):
        """Erstellt den Vorrat. WICHTIG: Kirchenteile zuerst!"""
        steine = []
        steine.append(Spielstein(spieler_id, 'Turm'))
        steine.append(Spielstein(spieler_id, 'Schiff'))
        for _ in range(7):
            steine.append(Spielstein(spieler_id, 'Haus'))
        return steine

    def initialisiere_spielfeld(self):
        """Setzt den Pfarrer in die Mitte. Rest ist leer."""
        self.draw_pfarrer(self.pfarrer_pos[0], self.pfarrer_pos[1])

    # ---------------------------------------------------------
    # LOGIK-KERN: BERECHNUNG DER ERLAUBTEN ZÜGE (VEKTOR)
    # ---------------------------------------------------------

    def hat_eigene_orthogonale_nachbarn(self, row, col, spieler_id):
        """Prüft, ob direkt daneben (N,O,S,W) ein eigener Stein steht."""
        nachbarn = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        for r, c in nachbarn:
            if 1 <= r <= ROWS and 1 <= c <= COLS:
                stein = self.spielfeld_matrix[r-1][c-1]
                if stein is not None and stein.spieler == spieler_id:
                    return True
        return False

    def get_valid_moves(self, spieler_id):
        """
        Gibt eine Liste (Vektor) aller erlaubten Aktionen zurück.
        Format: [{'type': 'place'|'move', 'pos'/'from'/'to': ..., ...}]
        """
        moves = []
        vorrat = self.unplatzierte_steine[spieler_id]

        # --- PHASE 1: PLATZIEREN (Vorrat nicht leer) ---
        if len(vorrat) > 0:
            naechster_stein = vorrat[0]

            # A) Kirchenteile (Turm/Schiff) -> Müssen in die Ecken
            if naechster_stein.typ in ['Turm', 'Schiff']:
                alle_ecken = [(1, 1), (1, 7), (7, 1), (7, 7)]

                # Partner-Ecken definieren (Diagonale)
                partner_map = {
                    (1, 1): (7, 7), (7, 7): (1, 1),
                    (1, 7): (7, 1), (7, 1): (1, 7)
                }

                for (r, c) in alle_ecken:
                    # Feld muss leer sein
                    if self.spielfeld_matrix[r-1][c-1] is None:
                        # Diagonal-Check: Ist die gegenüberliegende Ecke vom Gegner besetzt?
                        pr, pc = partner_map[(r, c)]
                        partner_stein = self.spielfeld_matrix[pr-1][pc-1]

                        diagonale_ok = True
                        if partner_stein is not None and partner_stein.spieler != spieler_id:
                            diagonale_ok = False # Gegner hat diese Diagonale schon

                        if diagonale_ok:
                            moves.append({'type': 'place', 'pos': (r, c), 'stone_type': naechster_stein.typ})

            # B) Häuser -> Freie Platzierung (aber nicht neben eigene Steine)
            else:
                for r in range(1, ROWS + 1):
                    for c in range(1, COLS + 1):
                        # Feld leer & nicht Pfarrer
                        if self.spielfeld_matrix[r-1][c-1] is None and (r, c) != self.pfarrer_pos:
                            # Regel: Nicht Kante-an-Kante mit eigenen Steinen
                            if not self.hat_eigene_orthogonale_nachbarn(r, c, spieler_id):
                                moves.append({'type': 'place', 'pos': (r, c), 'stone_type': 'Haus'})

        # --- PHASE 2: BEWEGEN (Vorrat leer) ---
        else:
            for r in range(1, ROWS + 1):
                for c in range(1, COLS + 1):
                    stein = self.spielfeld_matrix[r-1][c-1]
                    # Nur eigene Steine prüfen
                    if stein and stein.spieler == spieler_id:

                        # Ziel berechnen
                        dx, dy = get_bewegungs_vektor(stein.ausrichtung)
                        ziel_r, ziel_c = r + dy, c + dx

                        # Ist Ziel auf dem Brett?
                        if 1 <= ziel_r <= ROWS and 1 <= ziel_c <= COLS:
                            ziel_inhalt = self.spielfeld_matrix[ziel_r-1][ziel_c-1]

                            # KOLLISIONS-CHECK: Ziel muss LEER sein (None) und kein Pfarrer
                            if ziel_inhalt is None and (ziel_r, ziel_c) != self.pfarrer_pos:
                                moves.append({'type': 'move', 'from': (r, c), 'to': (ziel_r, ziel_c)})

        return moves

    # ---------------------------------------------------------
    # INTERAKTION (Handle Click)
    # ---------------------------------------------------------

    def handle_click(self, event):
        row, col = self.get_grid_coords(event.x, event.y)
        if row is None: return

        # 1. Hole alle erlaubten Züge für die aktuelle Situation
        valid_moves = self.get_valid_moves(self.aktueller_spieler)

        # Bestimme Phase (Platzieren oder Bewegen)
        is_placement_phase = any(m['type'] == 'place' for m in valid_moves)
        stein_in_zelle = self.spielfeld_matrix[row-1][col-1]

        # --- FALL A: PLATZIEREN ---
        if is_placement_phase:
            # Prüfen, ob der Klick ein gültiger Platzierungs-Zug ist
            move_match = None
            for move in valid_moves:
                if move['type'] == 'place' and move['pos'] == (row, col):
                    move_match = move
                    break

            if move_match:
                # Popup für Richtungswahl öffnen
                waehler = RichtungsWaehler(self.master)
                gewaehlte_richtung = waehler.ergebnis

                if gewaehlte_richtung:
                    # Stein aus Vorrat holen und platzieren
                    neues_teil = self.unplatzierte_steine[self.aktueller_spieler].pop(0)
                    neues_teil.ausrichtung = gewaehlte_richtung
                    self.platziere_stein(row, col, neues_teil)
                    self.wechsle_spieler()
                else:
                    messagebox.showinfo("Info", "Platzierung abgebrochen.")
            else:
                # Fehlermeldung generieren, warum es nicht geht
                self.zeige_platzierungs_fehler(row, col, valid_moves)

        # --- FALL B: BEWEGEN ---
        else:
            # 1. Stein auswählen
            if not self.ausgewaehlter_stein_pos:
                if stein_in_zelle and stein_in_zelle.spieler == self.aktueller_spieler:
                    # Prüfen ob dieser Stein beweglich ist
                    kann_ziehen = any(m['type'] == 'move' and m['from'] == (row, col) for m in valid_moves)
                    if kann_ziehen:
                        self.ausgewaehlter_stein_pos = (row, col)
                        self.markiere_stein(row, col, True)
                    else:
                        messagebox.showwarning("Blockiert", "Dieser Stein kann nicht ziehen (Weg versperrt oder Brettende).")

            # 2. Stein bewegen (Ziel wählen)
            else:
                start_r, start_c = self.ausgewaehlter_stein_pos

                # Klick auf sich selbst -> Auswahl aufheben
                if (row, col) == (start_r, start_c):
                    self.markiere_stein(start_r, start_c, False)
                    self.ausgewaehlter_stein_pos = None
                    return

                # Prüfen ob Bewegung gültig ist
                zug_valide = any(m['type'] == 'move' and m['from'] == (start_r, start_c) and m['to'] == (row, col) for m in valid_moves)

                if zug_valide:
                    self.fuehre_zug_aus(row, col)
                    self.ausgewaehlter_stein_pos = None
                    self.wechsle_spieler()
                else:
                    messagebox.showwarning("Ungültig", "Hierhin kann der Stein nicht ziehen.")

    def zeige_platzierungs_fehler(self, r, c, valid_moves):
        """Hilfsfunktion für bessere Fehlermeldungen."""
        # Was steht als nächstes an?
        if len(self.unplatzierte_steine[self.aktueller_spieler]) > 0:
            typ = self.unplatzierte_steine[self.aktueller_spieler][0].typ
            if typ in ['Turm', 'Schiff']:
                messagebox.showwarning("Regel", "Kirchenteile müssen auf eine freie Ecke gesetzt werden!\n(Wenn der Gegner eine Ecke hat, gehört ihm die ganze Diagonale).")
            elif self.spielfeld_matrix[r-1][c-1] is not None:
                pass # Feld besetzt
            elif self.hat_eigene_orthogonale_nachbarn(r, c, self.aktueller_spieler):
                messagebox.showwarning("Regel", "Du darfst Häuser nicht direkt neben deine eigenen Steine setzen (nur diagonal erlaubt).")
            else:
                messagebox.showinfo("Info", "Ungültiges Feld.")

    def fuehre_zug_aus(self, ziel_row, ziel_col):
        """Führt den Zug visuell und logisch aus."""
        start_row, start_col = self.ausgewaehlter_stein_pos
        stein = self.spielfeld_matrix[start_row-1][start_col-1]

        # Löschen
        self.delete(f"piece_{start_row}_{start_col}")
        self.delete(f"arrow_{start_row}_{start_col}")
        self.delete(f"label_{start_row}_{start_col}")

        # Verschieben
        self.spielfeld_matrix[start_row-1][start_col-1] = None
        self.spielfeld_matrix[ziel_row-1][ziel_col-1] = stein

        # Drehen
        stein.drehen()

        # Neuzeichnen
        self.draw_piece(ziel_row, ziel_col, stein)

    def platziere_stein(self, row, col, stein_obj):
        self.spielfeld_matrix[row-1][col-1] = stein_obj
        self.draw_piece(row, col, stein_obj)

    def wechsle_spieler(self):
        self.aktueller_spieler = 3 - self.aktueller_spieler
        self.update_title()

    def update_title(self):
        vorrat = self.unplatzierte_steine[self.aktueller_spieler]
        if len(vorrat) > 0:
            typ = vorrat[0].typ
            phase = f"Platzieren: {typ}"
        else:
            phase = "Bewegen"
        self.master.title(f"Lass die Kirche im Dorf | Am Zug: {self.spieler_farben[self.aktueller_spieler]} | Phase: {phase}")

    # ---------------------------------------------------------
    # VISUALISIERUNG & GRAFIK
    # ---------------------------------------------------------

    def draw_grid(self):
        for i in range(ROWS + 1):
            p = i * CELL_SIZE + OFFSET
            self.create_line(OFFSET, p, WINDOW_SIZE + OFFSET, p, width=LINE_WIDTH, fill="#404040")
            self.create_line(p, OFFSET, p, WINDOW_SIZE + OFFSET, width=LINE_WIDTH, fill="#404040")

    def draw_labels(self):
        for i in range(7):
            c = (i + 0.5) * CELL_SIZE + OFFSET
            self.create_text(c, OFFSET/2, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))
            self.create_text(OFFSET/2, c, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))

    def get_canvas_coords(self, row, col):
        return (col - 0.5) * CELL_SIZE + OFFSET, (row - 0.5) * CELL_SIZE + OFFSET

    def get_grid_coords(self, x, y):
        if x < OFFSET or y < OFFSET: return None, None
        c, r = int((x-OFFSET)//CELL_SIZE)+1, int((y-OFFSET)//CELL_SIZE)+1
        if 1<=r<=ROWS and 1<=c<=COLS: return r, c
        return None, None

    def draw_pfarrer(self, r, c):
        x, y = self.get_canvas_coords(r, c)
        rad = CELL_SIZE * 0.2
        self.create_oval(x-rad, y-rad, x+rad, y+rad, fill='#333', outline='black', width=2, tags="pfarrer")
        self.create_text(x, y, text="P", fill='white', font=('Arial', 14, 'bold'), tags="pfarrer_label")

    def draw_piece(self, r, c, stein):
        x_c, y_c = self.get_canvas_coords(r, c)
        farbe = stein.get_farbe()
        rad = CELL_SIZE * 0.38

        # --- FORMEN ---
        if stein.typ == 'Haus':
            # Quadrat
            self.create_rectangle(x_c - rad, y_c - rad, x_c + rad, y_c + rad,
                                  fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")

        elif stein.typ == 'Schiff':
            # Kreis
            self.create_oval(x_c - rad, y_c - rad, x_c + rad, y_c + rad,
                             fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")

        elif stein.typ == 'Turm':
            # Dreieck
            points = [x_c, y_c - rad, x_c + rad, y_c + rad, x_c - rad, y_c + rad]
            self.create_polygon(points, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")

        # --- PFEILE (Bidirektional) ---
        if stein.typ != 'Pfarrer':
            if stein.ausrichtung in ['N', 'S']:
                dx, dy = 0, 1 # Vertikal
            else:
                dx, dy = 1, 0 # Horizontal

            arrow_len = CELL_SIZE * 0.35
            self.create_line(x_c - dx*arrow_len, y_c - dy*arrow_len,
                             x_c + dx*arrow_len, y_c + dy*arrow_len,
                             arrow=tk.BOTH, width=4, arrowshape=(10, 12, 5),
                             fill='black', tags=f"arrow_{r}_{c}")

        # Kleines Label zur Info (optional, kann man auch weglassen für cleaneren Look)
        # self.create_text(x_c - rad + 10, y_c - rad + 10, text=stein.typ[0], font=('Arial', 8))

    def markiere_stein(self, r, c, active):
        if self.spielfeld_matrix[r-1][c-1]:
            outline = '#FFFF00' if active else 'black' # Gelb wenn aktiv
            width = 5 if active else 2
            self.itemconfig(f"piece_{r}_{c}", outline=outline, width=width)

# ==========================================
# MAIN
# ==========================================
if __name__ == '__main__':
    root = tk.Tk()
    root.title("Lass die Kirche im Dorf")
    root.geometry(f"{WINDOW_SIZE + 2*OFFSET + 20}x{WINDOW_SIZE + 2*OFFSET + 20}")
    spielbrett = Spielbrett(root)
    root.mainloop()